# WIAG Ordensbezeichnungen

Zuordnung der IDs in der Quelle in Göttingen; vk 2022-03-30

## Vorspann

Modul laden

In [1]:
wds_path = "../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise

In [6]:
using WiagDataSetup

In [7]:
Wds = WiagDataSetup

WiagDataSetup

In [8]:
using DataFrames, CSV, Dates

In [9]:
data_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data"

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data"

In [10]:
out_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"

In [11]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

`id_in_source` nachtragen

In [23]:
str_map = """
5309;1;OP;13
5310;2;OFM;7
5311;3;OPraem;14
5312;4;OT;22
5313;5;OSB;16
5314;6;OCist;5
5315;7;OESA;106
5316;8;OCarm;3
5317;9;CanA;1
5318;10;SJ;23
5319;11;OMel;12
5320;12;OSA;36
5321;13;OSM;41
5322;14;OFMObs;107
5323;15;OFMCap;8
5324;16;OFMConv;108
5325;17;OFMRec;109
5326;18;OCart;4
5327;19;OSH;61
5328;20;ORC;110
5329;21;Otheat;96
92259;22;CRVC;46""";

In [24]:
df_map = DataFrame(id = Int[], id_old = Int[], abbreviation = String[], id_new = Int[]);

In [25]:
c_map = split(str_map, "\n");
for row in c_map
    c_row = split(row, ";")
    if length(c_row) > 1
        id = parse(Int, c_row[1])
        id_old = parse(Int, c_row[2])
        abbr = c_row[3]
        id_new = parse(Int, c_row[4])
        push!(df_map, (id, id_old, abbr, id_new))
    end
end

In [19]:
using MySQL

In [49]:
table_name = "item";
file_name = joinpath(out_path, "$(table_name)_update.sql")
file = open(file_name, "w")
for row in eachrow(df_map)
    line = "UPDATE " * table_name *
    " SET id_in_source = " * string(row[:id_new]) * 
    " WHERE id = " * string(row[:id]) *
    " AND item_type_id = 7;"
    println(file, line)
end
close(file)

``` SQL
source "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql/item_update.sql"
```

Zusätzliche Orden aufnehmen

In [16]:
using CSV, DataFrames

In [17]:
file_name = joinpath(data_path, "mapping_orden_stapel.csv")
df_map_stapel = CSV.read(file_name, DataFrame, delim = ";");

In [18]:
df_map_stapel[10:15, :]

,altLabel_abk,Orde,id_order
,String7?,String63,Int64
1,OESG,ordinis Wilhellemitarum,6
2,OMel,ordinis sancti Johannis,12
3,OP,ordinis Predicatorum,13
4,OPraem,ordinis Premonstratencis,14
5,OPraem,ordinis Premonstratencium,14
6,OPraem,ordinis Premonstratensis,14


In [19]:
df_map_stapel_g = groupby(df_map_stapel, :id_order)

,altLabel_abk,Orde,id_order
,String7?,String63,Int64
1,OCarm,ordinis Carmelitarum,3
2,OCarm,ordinis Carmellitarum,3
,altLabel_abk,Orde,id_order
,String7?,String63,Int64
1,missing,ordinis Augustinensium,130
2,missing,ordinis Augustini,130
3,missing,ordinis sanct Augustini,130
4,missing,ordinis sancti Augustini,130


In [20]:
df_item_ro_in = similar(df_map_stapel, 0)

,altLabel_abk,Orde,id_order
,String7?,String63,Int64


In [21]:
for g in df_map_stapel_g
    push!(df_item_ro_in, g[1, :])
end;

In [40]:
df_item_ro_in

,altLabel_abk,Orde,id_order
,String7?,String63,Int64
1,OCarm,ordinis Carmelitarum,3
2,OCart,ordinis Carthusiensium,4
3,OCist,ordinis Cisterciensis,5
4,OESG,ordinis sancti Wilhelmi,6
5,OMel,ordinis sancti Johannis,12
6,OP,ordinis Predicatorum,13
7,OPraem,ordinis Premonstratencis,14
8,OSB,Benedictinorum ordinis,16
9,OSBirg,ordinis sancte Bregitte,17


In [26]:
df_item_ro_in = antijoin(df_item_ro_in, df_map, on = :id_order => :id_new)

,altLabel_abk,Orde,id_order
,String7?,String63,Int64
1,OESG,ordinis sancti Wilhelmi,6
2,OSBirg,ordinis sancte Bregitte,17
3,OSC,ordinis Cruciferorum,18
4,TOF,ordinis tercii,24
5,missing,canonicus Regularis,113
6,missing,ordinis fratrum Minorum,115
7,missing,canonicus,116
8,missing,ordinis Augustinensium,130


Einträge in `item`

In [28]:
df_item_ro = DataFrame()

""
""


In [29]:
insertcols!(df_item_ro,
    :item_type_id => 7,
    :comment => "import for Utrecht priests",
    :edit_status => "import",
    :created_by => 7,
    :date_created => now(),
    :changed_by => 7,
    :date_changed => now(),
    :lang => "de",
    :id_in_source => df_item_ro_in.id_order,
    :is_online => 0,
);

In [78]:
table_name = "item"
file_name = joinpath(out_path, "$(table_name)_religious_order.sql")
Wds.insert_sql(file_name, table_name, df_item_ro)

8

``` sql
source "C:\Users\georg\Documents\projekte-doc\Utrecht-Priester\data_sql\item_religious_order.sql"
```

Eintrag in `religious_order`

Lies aus `item` die Einträge für Orden aus einer Datei (aus der Datenbank auf vweb)  

In [13]:
item_type_id = 7

7

In [79]:
file_name = joinpath(data_path, "item_religious_order.csv")
df_item = CSV.read(file_name, DataFrame, delim=";");

In [46]:
size(df_item)

(47476, 16)

In [80]:
df_item_religious_order = subset(df_item, :item_type_id => ByRow(isequal(item_type_id)))
df_idx = select(df_item_religious_order, [:id, :id_in_source, :item_type_id]);

Alternativ: Lies die Werte für `item.id` aus der lokalen Datenbank

In [14]:
sql = "SELECT id, id_in_source, item_type_id
FROM item
WHERE item_type_id = $(item_type_id)";
df_idx = Wds.sql_df(sql);

In [31]:
transform!(df_item_ro_in, :id_order => ByRow(string) => :id_in_source);

In [32]:
df_ro_in = innerjoin(df_item_ro_in, df_idx, on = :id_in_source)

,altLabel_abk,Orde,id_order,id_in_source,id,item_type_id
,String7?,String63,Int64,String,Int32,Int32
1,OESG,ordinis sancti Wilhelmi,6,6,176167,7
2,OSBirg,ordinis sancte Bregitte,17,17,176168,7
3,OSC,ordinis Cruciferorum,18,18,176169,7
4,TOF,ordinis tercii,24,24,176170,7
5,missing,canonicus Regularis,113,113,176171,7
6,missing,ordinis fratrum Minorum,115,115,176172,7
7,missing,canonicus,116,116,176173,7
8,missing,ordinis Augustinensium,130,130,176174,7


In [33]:
columns = [
    :altLabel_abk => :abbreviation,
    :Orde => :comment,
    :id => :id,
]

3-element Vector{Pair{Symbol, Symbol}}:
 :altLabel_abk => :abbreviation
         :Orde => :comment
           :id => :id

In [34]:
table_name = "religious_order"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, select(df_ro_in, columns))

8